# 식의약헬스종합정보
---
- 작성자 : 안지은
- 식품의약안전처 데이터 : 키가 동일하게 발급되며, URL에서 동일하게 나타나는 부분이 많음 -> 모듈화하여 사용
- 식품의약품안전처_의료기기 GMP 품질심사신청 기본정보 : NO_OPENAPI_SERVICE_ERROR 발생

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(category,key):
    url = 'https://apis.data.go.kr/1471000/%s?serviceKey=%s&pageNo=1&type=json&cnsgnItemName='%(category,key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(category,key,page):
    
    url = 'https://apis.data.go.kr/1471000/%s?serviceKey=%s&pageNo=%s&numOfRows=100&type=json&cnsgnItemName='%(category, key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

## 1. 식품의약품안전처_묶음의약품정보서비스 
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 51454 개 행

In [4]:
drug_bundle = []
cnt = get_total_cnt('DrbBundleInfoService02/getDrbBundleList02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('DrbBundleInfoService02/getDrbBundleList02',key,i)
    for item in js:
        drug_bundle.append(item)


100%|██████████| 515/515 [09:14<00:00,  1.08s/it]


In [5]:
drug_bundle_df = pd.json_normalize(drug_bundle)
drug_bundle_df.columns = ['대표 품목기준코드','대표 제품명', '대표 주성분', '대표 함량', '대표 업소명','대표 제조소',
                        '대표 허가일자','대표 심평원주성분코드','대표 심평원제품코드','대표 ATC코드','취소/취하명','품목기준코드','제품명',
                        '업소명','제조소','허가일자','심평원제품코드','취소/취하명']
drug_bundle_df

,대표 품목기준코드,대표 제품명,대표 주성분,대표 함량,대표 업소명,대표 제조소,대표 허가일자,대표 심평원주성분코드,대표 심평원제품코드,대표 ATC코드,취소/취하명,품목기준코드,제품명,업소명,제조소,허가일자,심평원제품코드,취소/취하명
0,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
1,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
2,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
3,196900027,비타민케이1주사액(피토나디온주사액),피토나디온(비타민케이),10밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-03-11 00:00:00,248830BIJ,645101681,B02BA01 (phytomenadione),정상,202104151,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-25 00:00:00,None,정상
4,196900031,대한에피네프린주사액,에피네프린,1밀리그램,대한약품공업(주),대한약품공업(주) 안산1공장 제 1공장,1969-10-23 00:00:00,152630BIJ,645100741,C01CA24 (epinephrine),정상,202103993,,제일약품(주),대한약품공업(주) 안산1공장 제 1공장,2021-05-20 00:00:00,None,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51449,200403232,레보니아정(레보노르게스트렐),레보노르게스트렐,0.75밀리그램,명문제약(주),명문제약(주) 화성 제 1공장,2004-10-11 00:00:00,None,None,G03AD01 (levonorgestrel),유효기간만료,200600232,엠에스필정(레보노르게스트렐),태극제약(주),명문제약(주) 화성 제 1공장,2006-02-09 00:00:00,None,유효기간만료
51450,200403232,레보니아정(레보노르게스트렐),레보노르게스트렐,0.75밀리그램,명문제약(주),명문제약(주) 화성 제 1공장,2004-10-11 00:00:00,None,None,G03AD01 (levonorgestrel),유효기간만료,200600011,레보노민정(레보노르게스트렐),신풍제약(주),명문제약(주) 화성 제 1공장,2006-02-03 00:00:00,None,유효기간만료
51451,199900565,소아용록시틸과립(록시트로마이신),록시트로마이신,50밀리그램,코오롱제약(주),코오롱제약(주) 대전 제 1공장,1999-02-18 00:00:00,None,None,J01FA06 (roxithromycin),취하,200606692,아이록스과립(록시트로마이신),한올바이오파마(주),코오롱제약(주) 대전 제 1공장,2006-11-15 00:00:00,None,취하
51452,199900565,소아용록시틸과립(록시트로마이신),록시트로마이신,50밀리그램,코오롱제약(주),코오롱제약(주) 대전 제 1공장,1999-02-18 00:00:00,None,None,J01FA06 (roxithromycin),취하,200606692,아이록스과립(록시트로마이신),한올바이오파마(주),코오롱제약(주) 대전 제 1공장,2006-11-15 00:00:00,None,취하


## 2. 식품의약품안전처_마약류취급완료보고 사용내역 정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 13391개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [6]:
drugs = []
cnt = get_total_cnt('NarkTrmtCmptnRptUseDtilsService01/getNarkTrmtCmptnRptUseDtilsInq01',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('NarkTrmtCmptnRptUseDtilsService01/getNarkTrmtCmptnRptUseDtilsInq01',key,i)
    for item in js:
        drugs.append(item['item'])

100%|██████████| 134/134 [01:46<00:00,  1.26it/s]


In [7]:
drugs_df = pd.json_normalize(drugs)
drugs_df.columns = ['품명','수량','사용량','재고량','사용내역','구입처']
drugs_df

,품명,수량,사용량,재고량,사용내역,구입처
0,브롬화수소산덱스트로메토판,500,0,0,자사에서는 2006.1.8. 현재 표준품 분양을 받지 않았습니다.,예정 구입처 : 식품의약품의약품평가부 마약신경계의약품팀
1,브롬화수소산덱스트로메토판,500,0,0,1,1
2,브롬화수소산덱스트로메토판,500,100,400,2007-03-08 Lot S-0450207(200kg) 자가품질관리용으로 사용,식품의약품의약품평가부마약신경계의약품팀
3,브롬화수소산덱스트로메토판,500,100,400,2007-03-08 LOT S-0450207(200kg) 자가품질관리용으로 사용,식품의약품안전청평가부마약신경계의약품팀
4,브롬화수소산덱스트로메토판,400,100,300,2007-07-27 LOT S-1060407(100kg) 자가품질관리용으로 사용,식품의약품의약품평가부마약신경계의약품팀
...,...,...,...,...,...,...
13386,덱스트로메토르판브롬화수소산염수화물,10519.29,2272.82,8246.47,"상용표준품(S-0220222) : 2,272.82 mg",국전약품 상용표준품
13387,부프레노르핀,200,0,0,Buprenorphine impurity A \n취급승인양 : 200 mg\n수불한...,구입내역 없음
13388,부프레노르핀,15000,0,0,"취급승인양 : 15,000 mg\n수불한양 : -\n실제사용량 : -\n재고량 : -",구입내역 없음
13389,부프레노르핀 포 시스템 슈터빌리티,80,0,70,취급승인양 : 80mg\n수불한양 : 70mg\n실제사용량 : 0 mg\n재고량 :...,EP


## 3. 식품의약품안전처_연도별 마약류취급자 정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 13391개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [8]:
taker = []
cnt = get_total_cnt('YrNarkAgntStusService01/getYrNarkAgntStusInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('YrNarkAgntStusService01/getYrNarkAgntStusInq02',key,i)
    for item in js:
        taker.append(item['item'])

100%|██████████| 1/1 [00:00<00:00,  2.16it/s]


In [9]:
taker_df = pd.json_normalize(taker)
taker_df.columns = ['연도','건수']
taker_df

,연도,건수
0,1977,3
1,1989,2
2,1997,2
3,2005,11
4,2013,62
5,2021,264
6,1980,1
7,1990,1
8,1998,4
9,2006,28


## 4. 식품의약품안전처_의료기기 포장단위정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 102430개 행

In [10]:
pkgs = []
cnt = get_total_cnt('MdeqPcknUnitInfoService02/getMdeqPcknUnitInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqPcknUnitInfoService02/getMdeqPcknUnitInq02',key,i)
    for item in js:
        pkgs.append(item)

100%|██████████| 1025/1025 [14:33<00:00,  1.17it/s]


In [11]:
pkgs_df = pd.json_normalize(pkgs)
pkgs_df.columns = ['의료기기품목일련번호','기관지역구분','업종','허가신고구분',
                    '의료기기품목허가번호','품목명',
                    '허가일자','취소취하구분','취소취하일자','상품명',
                    '일련번호','포장단위명']
pkgs_df

,의료기기품목일련번호,기관지역구분,업종,허가신고구분,의료기기품목허가번호,품목명,허가일자,취소취하구분,취소취하일자,상품명,일련번호,포장단위명
0,106,서울청,제조업,신고,서울 제신 06-182 호,체외형범용프로브,2006-11-10,None,None,None,1,자사 포장단위에 의함
1,108,서울청,제조업,신고,서울 제신 06-183 호,의료용프로브(체외형),2006-11-10,None,None,None,1,자사 포장단위에 의함
2,110,서울청,수입업,신고,서울 수신 06-1679 호,의료용가이드,2006-11-10,None,None,None,1,제조원의 포장단위에 의함.
3,114,서울청,제조업,신고,서울 제신 06-181 호,의료용절삭기구,2006-11-10,None,None,Bonesier,1,자사 포장단위에 의함
4,1200200209,경인청,제조업,신고,경인 제신 03-25 호,부목,2003-03-07,None,None,None,1,1개
...,...,...,...,...,...,...,...,...,...,...,...,...
102425,70,부산청,수입업,신고,부산 수신 06-40 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102426,72,부산청,수입업,신고,부산 수신 06-38 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102427,74,부산청,수입업,신고,부산 수신 06-37 호,시력보정용안경렌즈,2006-11-07,None,None,None,1,1매 또는 1개
102428,76,의료기기정보기술지원센터,수입업,인증,수인 06-1144 호,근전도계,2006-11-07,None,None,NIM-Neuro 2.0 System,1,1UNIT/BOX


## 5. 식품의약품안전처_수입식품 해외제조업소 정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 96792개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [12]:
foreign_food = []
cnt = get_total_cnt('IprtFoodOvseaMnftBsshInfoService01/getIprtFoodOvseaMnftBsshInfoInq01',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('IprtFoodOvseaMnftBsshInfoService01/getIprtFoodOvseaMnftBsshInfoInq01',key,i)
    for item in js:
        foreign_food.append(item['item'])

 88%|████████▊ | 852/968 [12:31<01:58,  1.02s/it]

In [ ]:
foreign_food_df = pd.json_normalize(foreign_food)
foreign_food_df.columns = ['해외제조업소코드','해외제조업소명','해외제조업소영업구분코드','해외제조업소영업구분명',
                            '식품구분코드','식품구분명','시설취소일시','국가코드','국가명','지역코드','지역명','식품안전관리시스템인증여부',
                            '인증명','인증기관명','인증기관일시','인증기관만료일시',
                            '취하여부','취하일시','취소중단코드','취소중단명','수동등록구분코드',
                            '식품유통시작일시','식품유통종료일시','수산시작일시','수입중단번호']
foreign_food_df

## 6. 식품의약품안전처_의료기기 동등제품정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 13391개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [ ]:
equi_device = []
cnt = get_total_cnt('MdeqEqvlPrdtInfoService02/getMdeqEqvlPrdtInfoInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqEqvlPrdtInfoService02/getMdeqEqvlPrdtInfoInq02',key,i)
    for item in js:
        equi_device.append(item['item'])

In [ ]:
equi_device_df = pd.json_normalize(equi_device)
equi_device_df.columns = ['동등제품제조수입구분','동등제품허가년','동등제품허가번호',
                        '동등제품허가일련번호','동등공고번호','동등공고품목명','동등공공유형번호','동등공고날짜',
                        '동일성제품여부','동일성제품허가번호','동등공고제품','의료기기품목번호','기관지역구분','업종',
                        '허가신고구분','의료기기품목허가번호','품목명','허가일자','취소취하구분','취소취하일자','상품명']
equi_device_df

## 7. 식품의약품안전처_의료기기 품목정보 
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [ ]:
medi_device = []
cnt = get_total_cnt('MdeqPrdlstInfoService02/getMdeqPrdlstInfoInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqPrdlstInfoService02/getMdeqPrdlstInfoInq02',key,i)
    for item in js:
        medi_device.append(item['item'])

In [ ]:
medi_device_df = pd.json_normalize(medi_device)
medi_device_df.columns = ['의료기기품목일련번호','치수','특성명','제조방법','사용목적','사용방법',
                        '주의사항','유효기관','시험규격','기관지역구분','업종','허가신고구분','의료기기품목허가번호',
                        '품목명','허가일자','취소취하구분코드', '취소취하일자','상품명','분류번호','모델명','등급',
                        '제조의뢰자','제조자','제조국']
medi_device_df

## 8. 식품의약품안전처_의료기기 GMP 신청품목 현황
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 39746개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음


In [ ]:
gmp = []
cnt = get_total_cnt('MdeqGMPAplyPrdlstStusService02/getMdeqGMPAplyPrdlstStusInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqGMPAplyPrdlstStusService02/getMdeqGMPAplyPrdlstStusInq02',key,i)
    for item in js:
        gmp.append(item['item'])

In [ ]:
gmp_df = pd.json_normalize(gmp)
gmp_df.columns = ['최초심사여부','진행상태코드','진행상태','담당자승인여부','심사청 코드','심사청',
                    '심사방식코드','심사방식','적용기준구분코드','적용기준구분','품모경','대표품목유무',
                    '품목군제외여부','심사신청일련번호','심사신청품목일련번호','분류번호','등급','형명',
                    '접수번호','제조자국가코드','제조자국가','제조자 명','제조자 주소','상태','수출용 여부']
gmp_df

## 9. 식품의약품안전처_의료기기 회수폐기방법별결과 
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 445개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [ ]:
discard = []
cnt = get_total_cnt('MdeqReclDscdMthRsltService02/getMdeqReclDscdMthRsltInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqReclDscdMthRsltService02/getMdeqReclDscdMthRsltInq02',key,i)
    for item in js:
        discard.append(item['item'])

In [ ]:
discard_df = pd.json_normalize(discard)
discard_df.columns = ['회수방법대분류','회수방법소분류','회수폐기보고구분코드','회수폐기보고구분',
                    '보고승인일자','보고제출일자','업소명','대표자명','업종구분코드','업종구분명',
                    '업소주소','공개마감일','소비자가취해야할 행동','제품이미지ID1','제품이미지ID2',
                    '제품이미지ID3','보고상태코드','보고상태','회수폐기보고코드','회수폐기보고코드명']
                    
discard_df

## 10. 식품의약품안전처_의료기기 임상실시기관정보
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 413개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [ ]:
test = []
cnt = get_total_cnt('MedqClncCndctInstInfoService02/getMedqClncCndctInstInfoInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MedqClncCndctInstInfoService02/getMedqClncCndctInstInfoInq02',key,i)
    for item in js:
        test.append(item['item'])

In [ ]:
test_df = pd.json_normalize(test)
test_df.columns = ['의료기관의명칭','임상시험기관지정번호','지정일자','의료기관장또는개설자','의료기관구분(종별)',
                    '의료기관구분(구분)','요양기관지정구분','임상시험심사위원회구분','수련구분','임상시험실시기관일련번호',
                    '기관의우편번호','기관의주소1','기관의주소2','진료과목']
                    
test_df

## 11. 식품의약품안전처_의료기기 광고제작사별 심의내역
---
- 실행일 : 2023-01-10
- 한 페이지 당  100개 씩 제한
- 총 26750개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [ ]:
advertise = []
cnt = get_total_cnt('MdeqAdMakrDlbrDtilsService02/getMdeqAdMakrDlbrDtilsInq02',key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts('MdeqAdMakrDlbrDtilsService02/getMdeqAdMakrDlbrDtilsInq02',key,i)
    for item in js:
        advertise.append(item['item'])

In [ ]:
advertise_df = pd.json_normalize(advertise)
advertise_df.columns = ['광고업소명','대표품목명','결과통보_심의번호',
                        '결과통보_심의결과일자','결과통보_결과','결과통보_재심의여부']
advertise_df